# Data Aggregation

Avinash Bisram (8/29/24)

Now that we've...
* determined our data needs based on the KPIs and desired functionality of our final dashboard
* identified the data sources
* performed data quality assurance (validation) on each individual source

It's time to prepare the data to feed into the dashboard.

This includes
* performing any necessary joins
* new column calculations
* filtering out unneeded columns
* repeating those steps to facilitate visualizing all the metrics and trends that we want in the final dashboard.

## Assumed Data Relationships

![image](Images/Data%20Relationships.jpg)

## Metric Checklist

Below is a check-list of all the metrics we need from this data. As we perform the necessary joins to collect this data, we will edit this checklist and see what is remaining at the end (requiring additional data collection recommendations).

![image](Images/InitialDashboardMetricChecklist.jpg)

In [1]:
# Importing necessary packages

import numpy as np
import pandas as pd

In [11]:
# Sales pipeline has our key information at the deal level so let's load that first

df_sales_pipeline = pd.read_csv('./Data/sales_pipeline_CLEAN.csv')

df_sales_pipeline.info()
df_sales_pipeline.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
dtypes: float64(1), object(7)
memory usage: 550.1+ KB


,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0
5,ZNBS69V1,Anna Snelling,MG Special,Ron-tech,Won,2016-10-29,2017-03-01,49.0
6,9ME3374G,Vicki Laflamme,MG Special,J-Texon,Won,2016-10-30,2017-03-02,57.0
7,7GN8Q4LL,Markita Hansen,GTX Basic,Cheers,Won,2016-11-01,2017-03-07,601.0
8,OLK9LKZB,Niesha Huffines,GTX Plus Basic,Zumgoity,Won,2016-11-01,2017-03-03,1026.0
9,HAXMC4IX,James Ascencio,MG Advanced,NaN,Engaging,2016-11-03,NaN,NaN


## Join 1: Manager per deal

Next, we know this dashboard is a at a manager level overall so we want to find the manager attached to each deal.

To do this, we need to join this table with sales_team. Ideally, every sales_agent SHOULD have a valid manager so we can use an inner join however, to be thorough, we will do a left join first and perform two sanity checks:
* Check 1: Make sure every deal is present only once (hopefully there were no duplications due to this join)
* Check 2: Confirm that every row has a manager attached

In [15]:
# Load the sales_teams table

df_sales_teams = pd.read_csv('./Data/sales_teams.csv')

df_sales_teams.info()
df_sales_teams.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sales_agent      35 non-null     object
 1   manager          35 non-null     object
 2   regional_office  35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


,sales_agent,manager,regional_office
0,Anna Snelling,Dustin Brinkmann,Central
1,Cecily Lampkin,Dustin Brinkmann,Central
2,Versie Hillebrand,Dustin Brinkmann,Central
3,Lajuana Vencill,Dustin Brinkmann,Central
4,Moses Frase,Dustin Brinkmann,Central
5,Jonathan Berthelot,Melvin Marxen,Central
6,Marty Freudenburg,Melvin Marxen,Central
7,Gladys Colclough,Melvin Marxen,Central
8,Niesha Huffines,Melvin Marxen,Central
9,Darcel Schlecht,Melvin Marxen,Central


In [28]:
# Double checking that every sales agent in this table only has ONE manager

# Grouping by sales agent and counting managers
temp = df_sales_teams.groupby(by = 'sales_agent').agg('count')

# Returning how many records have a manager count more than 1
print(f"Number of agents with more than 1 manager: {len(temp[temp['manager'] > 1])}")

Number of agents with more than 1 manager: 0


In [34]:
# Going ahead with the join

df_deals = df_sales_pipeline.join(other = df_sales_teams.drop('regional_office', axis=1).set_index('sales_agent')
                                  , how = 'left'
                                  , on = 'sales_agent')

df_deals.info()
df_deals.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
 8   manager         8800 non-null   object 
dtypes: float64(1), object(8)
memory usage: 618.9+ KB


,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,manager
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,Dustin Brinkmann
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0,Melvin Marxen
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,Melvin Marxen
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,Dustin Brinkmann
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,Summer Sewald
5,ZNBS69V1,Anna Snelling,MG Special,Ron-tech,Won,2016-10-29,2017-03-01,49.0,Dustin Brinkmann
6,9ME3374G,Vicki Laflamme,MG Special,J-Texon,Won,2016-10-30,2017-03-02,57.0,Celia Rouche
7,7GN8Q4LL,Markita Hansen,GTX Basic,Cheers,Won,2016-11-01,2017-03-07,601.0,Celia Rouche
8,OLK9LKZB,Niesha Huffines,GTX Plus Basic,Zumgoity,Won,2016-11-01,2017-03-03,1026.0,Melvin Marxen
9,HAXMC4IX,James Ascencio,MG Advanced,NaN,Engaging,2016-11-03,NaN,NaN,Summer Sewald


In [36]:
# Sanity Check # 1: Comparing the length of new dataframe to original

print(f"Same record count before and after join? {len(df_deals) == len(df_sales_pipeline)}")

Same record count before and after join? True


In [38]:
# Sanity Check # 2: No null managers?

print(f"Number of missing managers: {df_deals['manager'].isnull().sum()}")

Number of missing managers: 0


Looks like the join to get the manager per deal was a success!

## Join 2: Product Series and COGS per deal

Next we need to know the cost of goods sold per deal so we can aggregate for manager and also calculate net revenue.

This means we need to perform another join with the products table.

The relationship between products and sales_pipeline SHOULD be 1 to many so if we perform a left join with sales_pipeline, we should expect the number of rows to remain unchanged.

Sanity Checks:
* Same number of rows before and after join
* Every deal has a COGS
* Every deal has a product "series"

In [39]:
# Loading the table

df_products = pd.read_csv('./Data/products_CLEAN.csv')

df_products.info()
df_products.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product      7 non-null      object
 1   series       7 non-null      object
 2   sales_price  7 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


,product,series,sales_price
0,GTX Basic,GTX,550
1,GTX Pro,GTX,4821
2,MG Special,MG,55
3,MG Advanced,MG,3393
4,GTX Plus Pro,GTX,5482
5,GTX Plus Basic,GTX,1096
6,GTK 500,GTK,26768


In [41]:
# Performing the join (storing it in temp before sanity checks)

temp = df_deals.join( other = df_products.set_index('product')
                    , how = 'left'
                    , on = 'product')

temp.head(10)

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,manager,series,sales_price
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,Dustin Brinkmann,GTX,1096
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0,Melvin Marxen,GTX,4821
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,Melvin Marxen,MG,55
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,Dustin Brinkmann,GTX,550
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,Summer Sewald,GTX,550
5,ZNBS69V1,Anna Snelling,MG Special,Ron-tech,Won,2016-10-29,2017-03-01,49.0,Dustin Brinkmann,MG,55
6,9ME3374G,Vicki Laflamme,MG Special,J-Texon,Won,2016-10-30,2017-03-02,57.0,Celia Rouche,MG,55
7,7GN8Q4LL,Markita Hansen,GTX Basic,Cheers,Won,2016-11-01,2017-03-07,601.0,Celia Rouche,GTX,550
8,OLK9LKZB,Niesha Huffines,GTX Plus Basic,Zumgoity,Won,2016-11-01,2017-03-03,1026.0,Melvin Marxen,GTX,1096
9,HAXMC4IX,James Ascencio,MG Advanced,NaN,Engaging,2016-11-03,NaN,NaN,Summer Sewald,MG,3393


In [42]:
# Sanity Check # 1: Same number of records before and after

print(f"Same record count before and after join? {len(df_deals) == len(temp)}")

Same record count before and after join? True


In [43]:
# Sanity Check # 2: No null series

print(f"Number of missing product series: {temp['series'].isnull().sum()}")

Number of missing product series: 0


In [44]:
# Sanity Check # 3: No null cogs

print(f"Number of missing product series: {temp['sales_price'].isnull().sum()}")

Number of missing product series: 0


All sanity checks were passed so we can save the current version of our aggregated data.

We will also change the name of sales price to "cogs" to make it easier to understand.

In [45]:
# Saving the current version of our data

df_deals = temp

df_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
 8   manager         8800 non-null   object 
 9   series          8800 non-null   object 
 10  sales_price     8800 non-null   int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 756.4+ KB


In [47]:
# Renaming sales_price

df_deals = df_deals.rename({'sales_price' : 'cogs'}, axis = 1)

df_deals.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,manager,series,cogs
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,Dustin Brinkmann,GTX,1096
1,Z063OYW0,Darcel Schlecht,GTX Pro,Isdom,Won,2016-10-25,2017-03-11,4514.0,Melvin Marxen,GTX,4821
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,Melvin Marxen,MG,55
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,Dustin Brinkmann,GTX,550
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,Summer Sewald,GTX,550


## Note about the accounts table

**Note:** The final table, accounts, is not needed to calculate the key metrics relating to team performance but would be useful information to have if we later decide to implement an individual deal view to drill down on individual deals performed by a member of a team. For now, we won't join it to our data but we can always do so later in our dashboarding software since it is already prepared.

We will do one sanity check though to ensure that there are no duplicate "accounts" (the assumed Primary Key) in that table.

In [52]:
# Loading the accounts table

df_accounts = pd.read_csv('./Data/accounts_CLEAN.csv')

df_accounts.info()
df_accounts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.8+ KB


,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,Acme Corporation,technology,1996,1100.04,2822,United States,NaN
1,Betasoloin,medical,1999,251.41,495,United States,NaN
2,Betatech,medical,1986,647.18,1185,Kenya,NaN
3,Bioholding,medical,2012,587.34,1356,Philippines,NaN
4,Bioplex,medical,1991,326.82,1016,United States,NaN


In [53]:
# Checking if "account" column is unique

len(df_accounts) == len(df_accounts['account'].unique())

True

In [54]:
# Also checking if there are any null values in "account" because that would mess up a left join later on

df_accounts['account'].isnull().sum()

# Great

0

## Exporting the Aggregated Data

Below is the updated checklist identifying the desired metrics that are left unsatisfied with the current dataset. The key areas for further recommendation is that the current sales pipeline does not record when a sales agent begins prospecting a deal. Furthermore, there are no customer satisfaction ratings currently in the pipeline.

![image](Images/FinalDashboardMetricChecklist.jpg)

We can at least carry on with the net revenue view and some of the efficiency view but using our current data, we will not be able to visualize performance metric relating to customer satisfaction.

In [55]:
# Exporting the final data for visualization

df_deals.to_csv('./Data/aggregated_sales_data.csv', index = False)